In [18]:
import pandas as pd
import re

In [69]:
data = pd.read_excel('data.xlsx')
data

,address,price,properties
0,"The Ashland | 250 Ashland Pl, Brooklyn, NY","$4,269+ 1 bd","$8,015+ 2 bds$9,621+ 3 bds"
1,"Avalon Willoughby Square | 214 Duffield St, Br...","$3,374+ Studio","$3,864+ 1 bd"
2,"Ocean Drive | 1 Ocean Dr, Brooklyn, NY","$2,478+ Studio","$2,708+ 1 bd$3,125+ 2 bds"
3,"AVA Fort Greene | 343 Gold St, Brooklyn, NY","$3,116+ Studio","$3,130+ 1 bd$6,679+ 2 bds$7,963+ 3 bds"
4,"180 Montague | 180 Montague St, Brooklyn, NY","$4,216+ 1 bd",NaN
...,...,...,...
176,"229 Washington Ave, Brooklyn, NY","$5,500+ 3 bds","$6,499+ 4 bds"
177,"250 Bedford Ave, Brooklyn, NY","$3,900+ Studio","$4,850+ 1 bd"
178,"150 Ocean Ave APT 4F, Brooklyn, NY 11225","$1,950/mo",1 bd 1 ba -- sqft - Apartment for rent
179,"8789 23rd Avenue #, Brooklyn, NY","$2,000+ 2 bds","$2,800+ 3 bds"


In [70]:
new_data = pd.DataFrame(columns=["ApartmentID", "Price", "NumberOfBedrooms", "NumberOfBathrooms", "Address"])
new_data

,ApartmentID,Price,NumberOfBedrooms,NumberOfBathrooms,Address


In [72]:
new_index = 0

for index, row in data.iterrows():
    if ("bd" in row["price"] or "bds" in row["price"]):
        # Extract the prices
        prices = re.findall(r'\$([\d,]+)\+', row["price"])
        
        # Extract the number of bedrooms
        bedrooms = re.findall(r'(\d+)\s*bd', row["price"])
        
        if (row["properties"] is not None and isinstance(row["properties"], str)):
            new_prices = re.findall(r'\$([\d,]+)\+', row["properties"])
            
            prices.extend(new_prices)
            
            new_bedrooms = re.findall(r'(\d+)\s*bd', row["properties"])
            
            bedrooms.extend(new_bedrooms)
        
        for i in range(len(prices)):
            
            # Removing , from price
            prices[i] = prices[i].replace(",", "")
            
            # Because it's an apartment so it is guarantee to have at least one bathroom
            bathroom = 1
            
            # Inserting new row to the processed data
            new_data.loc[new_index] = [new_index+1, int(prices[i]), int(bedrooms[i]), bathroom, row["address"]]
            new_index += 1
            
    elif ("Studio" in row["price"]):
        # Extract the prices
        prices = re.findall(r'\$([\d,]+)\+', row["price"])
        
        bedrooms = [1]
        
        if (row["properties"] is not None and isinstance(row["properties"], str)):
            new_prices = re.findall(r'\$([\d,]+)\+', row["properties"])
            
            prices.extend(new_prices)
            
            new_bedrooms = re.findall(r'(\d+)\s*bd', row["properties"])
            
            bedrooms.extend(new_bedrooms)
        
        for i in range(len(prices)):
            
            # Removing , from price
            prices[i] = prices[i].replace(",", "")
            
            # Studio type apartment only have one bathroom and bedroom
            bathroom = 1
            bedroom = 1
            
            # Inserting new row to the processed data
            new_data.loc[new_index] = [new_index+1, int(prices[i]), int(bedrooms[i]), bathroom, row["address"]]
            new_index += 1
    
    else:
        # Replace all unnecessary character in price
        price = row["price"].replace("/mo", "")
        price = price.replace(",", "")
        price = price.replace("$", "")
        
        # Split the string based on variations of 'bds', 'bd', 'ba', and 'bas'
        pattern = r'(\d+)\s+(bds|bd|ba|bas)\b'
        hasil = re.findall(pattern, row["properties"])
        
        result = {}
        for item in hasil:
            if item[1] in result:
                result[item[1]] = int(result[item[1]]) + int(item[0])
            else:
                result[item[1]] = int(item[0])
        
        bedroom = 0
        bathroom = 0
        
        # Get the value of how many bedroom and bathroom at the apartment
        if "bd" in result:
            bedroom = result["bd"]
        elif "bds" in result:
            bedroom = result["bds"]
        
        if "ba" in result:
            bathroom = result["ba"]
        elif "bas" in result:
            bathroom = result["bas"]
        
        # Inserting new row to the processed data
        new_data.loc[new_index] = [new_index+1, int(price), bedroom, bathroom, row["address"]]
        new_index += 1
        
new_data

,ApartmentID,Price,NumberOfBedrooms,NumberOfBathrooms,Address
0,1,4269,1,1,"The Ashland | 250 Ashland Pl, Brooklyn, NY"
1,2,8015,2,1,"The Ashland | 250 Ashland Pl, Brooklyn, NY"
2,3,9621,3,1,"The Ashland | 250 Ashland Pl, Brooklyn, NY"
3,4,3374,1,1,"Avalon Willoughby Square | 214 Duffield St, Br..."
4,5,3864,1,1,"Avalon Willoughby Square | 214 Duffield St, Br..."
...,...,...,...,...,...
247,248,4850,1,1,"250 Bedford Ave, Brooklyn, NY"
248,249,1950,1,1,"150 Ocean Ave APT 4F, Brooklyn, NY 11225"
249,250,2000,2,1,"8789 23rd Avenue #, Brooklyn, NY"
250,251,2800,3,1,"8789 23rd Avenue #, Brooklyn, NY"


In [74]:
# Save PreProcessed Data to Excel
new_data.to_excel('proccessed_data.xlsx', index=False)